# Домашнее задание 3

Это домашнее задание по материалам второго семинаров. Дедлайн по отправке - 23:55 24 февраля. 

Домашнее задание выполняется в этом же Jupyter Notebook'e и присылается мне на почту: __beznosikov.an@phystech.edu__.

Решение каждой задачи необходимо поместить после её условия.

Файл должен называться: Фамилия_Имя_Optimization_HW_3

При полном запуске Вашего решения (Kernel -> Restart & Run All) все ячейки должны выполняться без ошибок. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import time 
np.random.seed(43)

## Задача 1

Вновь рассмотрим задачу минимизации эмпирического риска:
\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n l (g(w, x_i), y_i).
\end{equation}

В прошлом задании работа шла с линейной модель $g(w, x) = w^T x$ и квадратичную функцию потерь $l(z, y) = (z-y)^2$. 

__(а)__ В дополнение к квадратичной функции потерь реализуйте логистическую/сигмоидную: $l(z,y) = \ln (1 + \exp(-yz))$ (__Важно: $y$ должен принимать значения $-1$ или $1$__). Выпишите градиент. Является ли новая задача регресии выпуклой? Оцените $L$ для новой функции потерь. 

Имеем следующую функцию эмпирического риска: $$f(w) = \dfrac{1}{n} \sum\limits_{i=1}^n \ln\left(1 + \exp (-y_i w^Tx^i)\right)$$
Тогда ее градиент: $$\nabla f(w) = \dfrac{1}{n}\sum\limits_{i=1}^n \dfrac{-y_i \exp(-y_i w^T x^i)}{1 + \exp(-y_i w^T x^i)} \cdot x^i$$
А гессиан: $$ \nabla^2 f(w) = \dfrac{1}{n} \sum\limits_{i=1}^n \dfrac{\exp(-y_i w^Tx^i)}{(1 + \exp(-y_i w^Tx^i))^2} \cdot x^i(x^i)^T$$
Имеем: $\forall x^i \in \mathbb{R}^d \hookrightarrow x^i(x^i)^T \succeq 0$ и экспонента неотрицательна $\Rightarrow$ задача является выпуклой.

Константу липшица оценим следующим образом: $\forall x \hookrightarrow \dfrac{e^x}{(1 + e^x)^2} \leq \dfrac{1}{4}$, поэтому: $$\nabla^2 f(w) \preccurlyeq \dfrac{1}{4n} \sum\limits_{i=1}^n x^i(x^i)^T $$
Тогда константа L: 
$$L = \dfrac{1}{4n} \cdot \lambda_{max}(\sum\limits_{i=1}^n x^i(x^i)^T)$$

__(б)__ Возьмите датасет _mushrooms_ из прошлого задания. Проделайте следующие шаги из прошлого задания, только с логистической функцией потерь:

1) Разделите данные на две части: обучающую и тестовую.

2) Для обучающей части $X_{train}$, $y_{train}$ оцените константу $L$ задачи обучения/оптимизации.

3) Используя градиентный спуск, обучите новую модель (без ограничений и регуляризаций). Постройте график: точность от номера итерации.

4) Если в пункте 3) пришлось столкнуться с проблемами или просто необходимо улучшить точность, то добавьте ограничения или $\ell_2$-регуляризацию, как в прошлом ДЗ.

5) Сравните с результатами квадратичной функции потерь из прошлого ДЗ.

In [2]:
#Подгружаем данные
dataset = "mushrooms.txt" 
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]

In [3]:
#Меняем двойки на нули
y = pd.DataFrame(y)
y = np.array(y.replace(2, 0)).reshape(8124,)

In [4]:
#Класс логрегрессии
def sigmoid(h):
    return 1. / (1 + np.exp(-h))

class MyLogisticRegression(object):
    
    def __init__(self):
        self.w = None
    
    def fit(self, X, y, lr, lambd = False, eps = 10**(-3)):
        
        n, k = X.shape
        
        if self.w is None:
            self.w = np.random.randn(k)
        
        grad = np.random.randn(k)
        
        losses = []
        
        self.accuracy = []
        self.accuracy.append(self.w)
        
        while np.linalg.norm(grad, ord = 2) > eps:
            
            z = sigmoid(X @ self.w)
            
            grad = self._calc_grad(X, y, z, lambd)
            
            if len(losses) % 150 == 0:
                self.accuracy.append(self.w - lr * grad)
            
            self.w -= lr * grad
            
            losses.append(self._loss(y, z))

        return losses
        
    def _calc_grad(self, X, y, z, lambd):
        
        if bool(lambd) == True :
            lambdaI = 2 * lambd * np.eye(self.w.shape[0])
            grad = np.dot(X.T, (z - y)) / len(y) + lambdaI @ self.w
            
        grad = np.dot(X.T, (z - y)) / len(y)  
        
        return grad 
    
    def predict_proba(self, X):
        return sigmoid(X @ self.w)

    def predict(self, X, threshold = 0.5):
        return self.predict_proba(X) >= threshold
    
    def get_accuracy(self, X, y):
        accuracy = []
        for i in range(len(self.accuracy)):
            accuracy.append(accuracy_score(np.round(sigmoid(X @ self.accuracy[i])), y))
        
        return accuracy
      
    def _loss(self, y, p):
        p = np.clip(p, 1e-10, 1 - 1e-10)
        return np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))

In [5]:
#Делим данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43)

In [6]:
#Оцениваем L:
summ = 0
for i in range(len(X_train)):
    summ += X_train[i][np.newaxis]  @ X_train[i][np.newaxis].T
L = 1 / (4 * X_train.shape[0]) * np.max(np.linalg.eigvals(summ))
print('Константа липшица данной задачи: ', L)

Константа липшица данной задачи:  5.25


In [ ]:
%%time
#Смотрим точность по метрике accuracy_score. Это логично, тк мы решаем задачу классификации.
regressor = MyLogisticRegression()
regressor.fit(X_train, y_train, 1/L, eps = 10**(-2))
accuracy1 = regressor.get_accuracy(X_test, y_test)
print(r'Точность по метрике accuracy_score при epsilon = 0.01: ', accuracy_score(y_test, regressor.predict(X_test)))
regressor.fit(X_train, y_train, 1/L, eps = 10**(-3))
accuracy2 = regressor.get_accuracy(X_test, y_test)
print(r'Точность по метрике accuracy_score для epsilon = 0.001: ', accuracy_score(y_test, regressor.predict(X_test)))

Точность по метрике accuracy_score при epsilon = 0.01:  0.9901538461538462


Как видно, точность получилась хорошая, скорость сходимости тоже. Проблем с пунктом 3 не возникло.

In [ ]:
#Чтобы сравнить в пункте д с предыдущим заданием, тк там я пользовался r2_score
regressor = MyLogisticRegression()
regressor.fit(X_train, y_train, 1/L, eps = 10**(-2))
print(r'Точность по метрике r2_score при epsilon = 0.01: ', r2_score(y_test, regressor.predict(X_test)))
regressor.fit(X_train, y_train, 1/L, eps = 10**(-3))
print(r'Точность по метрике r2_score при epsilon = 0.001: ', r2_score(y_test, regressor.predict(X_test)))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (15, 6))
x1 = 150 * np.array(range(len(accuracy1)))
x2 = 150 * np.array(range(len(accuracy2)))
axs[0].plot(x1[1:], accuracy1[1:], label = r'$||\;\nabla f\;||_2 < 10^{-2} $')
axs[0].set_title("Зависимость точности предсказаний от номера итерации")
axs[0].set_ylabel("Точность по метрике accuracy_score")
axs[0].set_xlabel('Номер итерации')
axs[0].grid(alpha = 0.2)
axs[0].legend(loc = 'center right')

axs[1].plot(x2[1:], accuracy2[1:], label = r'$||\;\nabla f\;||_2 < 10^{-3} $')
axs[1].set_title("Зависимость точности предсказаний от номера итерации")
axs[1].set_ylabel("Точность по метрике accuracy_score")
axs[1].set_xlabel('Номер итерации')
axs[1].grid(alpha = 0.2)
axs[1].legend(loc = 'center right')

plt.show()

Первое значимое отличие от обычной линейной регрессии в количестве итераций: при использованиии данной модели до точности $0.01$ получается примерно одинаковое число итераций и чуть хуже r2_score ($\approx 0.99$ против $\approx 0.97$). При одинаковом же значении эпсилон в критерии получается около $18000$ итераций, но и r2_score = $1.0$ уже на $11000$, чего не получалось добиться в предыдущем дз.

## Задача 2

__(a)__ Реализуйте метод тяжелого шарика. 

In [ ]:
class MyHeavyBall(MyLogisticRegression):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs) # передаем именные параметры родительскому конструктору
        self.w = None
    
    def fit(self, X, y, lr, beta, lambd = False, eps = 10**(-2)):
        
        if self.w is None: self.w = np.random.randn(2, X.shape[1])
        
        grad = np.random.randn(X.shape[1])
        
        losses = []
        
        self.times = [0]
        time_now = 0
        
        self.accuracy = []
        self.accuracy.append(self.w[0])
        
        while np.linalg.norm(grad, ord = 2) > eps:
            
            start_time = time.time()
            
            z = sigmoid(X @ self.w[0])
            
            grad = self._calc_grad(X, y, z, lambd)
            
            if len(losses) % 100 == 0:
                self.accuracy.append(self.w[0] - lr * grad + beta * (self.w[0] - self.w[1]))
                
            crutch = self.w[0]
            
            self.w[0] -= lr * grad + beta * (self.w[0] - self.w[1])
            
            self.w[1] = crutch
            
            losses.append(self._loss(y, z))
            
            end_time = time.time()
            
            time_now += end_time - start_time
            
            if len(losses) % 100 == 0:
                self.times.append(time_now)

        return self
    
    def predict_proba(self, X):
        return sigmoid(X @ self.w[0])
    
    def get_times(self):
        return self.times
    
    def get_accuracy(self, X, y):
        accuracy = []
        for i in range(len(self.accuracy)):
            if i != 0 and accuracy_score(np.round(sigmoid(X @ self.accuracy[i])), y) == 1.0:
                break
            accuracy.append(accuracy_score(np.round(sigmoid(X @ self.accuracy[i])), y))
        
        return accuracy

In [ ]:
%%time
#Смотрим точность по метрике accuracy_score. Это логично, тк мы решаем задачу классификации.
regressor = MyHeavyBall()
regressor.fit(X_train, y_train, 1/L, -0.77777778, eps = 10**(-2))
accuracy1 = regressor.get_accuracy(X_test, y_test)
print(r'Точность по метрике accuracy_score при epsilon = 0.01: ', accuracy_score(y_test, regressor.predict(X_test)))

__(б)__ Решите задачу логистической регрессии с помощью метода тяжелого шарика (не забудьте разделить выборку на две части: обучающую и тестовую). Зафиксируйте шаг $\frac{1}{L}$ и перебирайте разные значения моментума от -1 до 1. Постройте график сходимости метода от числа итераций (критерий сходимости подберите самостоятельно) для различных значений моментума. Всегда ли сходимость является монотонной?

Используем стандартный критерий $\| \nabla f\| < 10^{-2}$.

In [ ]:
#GridSearch по значениям моментума
Betas = np.linspace(-1, 1, 9)
accuracy = []
x = []
for i in range(len(Betas)):
    regressor = MyHeavyBall()
    regressor.fit(X_train, y_train, 1/L, Betas[i])
    accuracy.append(regressor.get_accuracy(X_test, y_test))
    x.append(100 * np.array(range(len(accuracy[i]))))

In [ ]:
fig, axs = plt.subplots(3, 3, figsize = (15, 15))
fig.suptitle(r'Графики сходимости для различных значений $\beta$', y = 0.93, fontsize = 30)
k = 0 
for i in range(3):
    for j in range(3):
        axs[i][j].plot(x[k][1:], accuracy[k][1:], label = r'$\beta = $' + str(np.round(Betas[k], 2)))
        axs[i][j].set_ylabel("accuracy_score")
        axs[i][j].set_xlabel('Номер итерации')
        axs[i][j].grid(alpha = 0.2)
        axs[i][j].legend(loc = 'center right')
        k += 1
plt.show()

__(в)__ Для лучшего значения моментума постройте график зависимости точности модели на тестовой выборке от времени работы метода. Добавьте на этот же график сходимость градиентного спуска с шагом $\frac{1}{L}$. Сделайте вывод.

Как видно из графиков, лучшее значение моментума получилось $\beta = 0.75$, посмотрим на скорость:

In [ ]:
class My_Linear_Regression:
    def __init__(self, fit_intercept = True):
        self.fit_intercept = fit_intercept
    
    def fit(self, X, y, gamma, eps = 10**(-3)):
        n, k = X.shape
        
        self.w = np.random.randn(k + 1 if self.fit_intercept else k)
        
        X = np.hstack((X, np.ones((n, 1)))) if self.fit_intercept else X
        
        self.times = [0]
        time_now = 0
        
        self.accuracy = []
        self.accuracy.append(self.w)
        self.losses = []
        grad = np.random.randn(k + 1 if self.fit_intercept else k)
        
        self.grads = [np.linalg.norm(grad, ord = 2)]
        while np.linalg.norm(grad, ord = 2) > eps:
            
            start_time = time.time()
            
            y_pred = self.predict(X)
            self.losses.append(MSE(y_pred, y))

            grad = self._calc_gradient(X, y, y_pred)
            self.grads.append(np.linalg.norm(grad, ord = 2))
            assert grad.shape == self.w.shape, f"gradient shape {grad.shape} is not equal weight shape {self.w.shape}"
            
            self.w -= gamma * grad
            
            if len(self.grads) % 100 == 0: 
                #print(f'Занчение нормы градиента на {len(self.grads)} итерациии - ', np.linalg.norm(grad, ord = 2))
                self.accuracy.append(self.w - gamma * grad)
                
            end_time = time.time()
            
            time_now += end_time - start_time
            
            if len(self.grads) % 100 == 0:
                self.times.append(time_now)
            
        return self
    
    def predict(self, X):
        
        n, _ = X.shape
        
        if self.fit_intercept:
            X = np.hstack((X, np.ones((n, 1))))
        
        assert X.shape[1] == self.w.shape[0], f"жопа в предикте"
        
        y_pred = X @ self.w

        return y_pred
    
    def _calc_gradient(self, X, y, y_pred):
        
        n, _ = X.shape
        
        if self.fit_intercept:
            X = np.hstack((X, np.ones((n, 1))))
        
        grad = (2 / n) * (X.T @ (y_pred - y))
        
        return grad
    
    def get_losses(self):
        
        return self.losses
    
    def get_accuracy(self, X, y):
        accuracy = []
        for i in range(len(self.accuracy)):
            if i != 0 and accuracy_score(np.round(X @ self.accuracy[i]) , y) == 1.0:
                break
            accuracy.append(accuracy_score(np.round(X @ self.accuracy[i]) , y))
        
        return accuracy
    
    def get_grads(self):
        
        return self.grads
    
    def get_times(self):
        return self.times

In [ ]:
Lin_reg = My_Linear_Regression(fit_intercept = False)
Heavy = MyHeavyBall()
Lin_reg.fit(X_train, y_train, 0.095725, eps = 10**(-3))
Heavy.fit(X_train, y_train, 1/L, 0.75, eps = 10**(-3))
fig = plt.figure(figsize = (15, 10))

plt.plot(Lin_reg.get_times()[:len(Lin_reg.get_accuracy(X_test, y_test))], Lin_reg.get_accuracy(X_test, y_test), 
         color = 'red', label = 'Обычная линейная регрессия')

plt.plot(Heavy.get_times()[:len(Heavy.get_accuracy(X_test, y_test)[:-1])], Heavy.get_accuracy(X_test, y_test)[:-1],
         color = 'black',  label = 'Тяжелый шарик')

plt.title("Зависимость accuracy от времени работы")
plt.xlabel("Время, сек")
plt.ylabel("Точность по метрике accuracy_score")

plt.legend(loc = 'center right')
plt.grid(alpha = 0.2)
plt.show()

Как видно, получилось, что тяжелый шарик работает дольше, чем обычная линейная регрессия, но, как было показано ранее, может показать лушее значение accuracy_score для заданного значения критерия.

__(г)__ Если в пунктах (б) и (в) столкнулись с проблемами, попробуйте $\ell_2$-регуляризовать задачу или рассмотреть ее на ограниченном множестве.

С проблемами не столкнулся.

__(д)__ Реализуйте ускоренный метод Нестерова (в форме Нестерова, а не который доказывали на семинаре). 

In [ ]:
class Nesterov(MyLogisticRegression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs) # передаем именные параметры родительскому конструктору
        self.w = None
    
    def fit(self, X, y, lr, beta, const = 0, lambd = False, eps = 10**(-2)):
        
        if self.w is None: self.w = np.random.randn(2, X.shape[1])
        
        grad = np.random.randn(X.shape[1])
        
        losses = []
        losses.append(np.linalg.norm(grad, ord = 2))
        
        i = 0
        
        self.times = [0]
        time_now = 0
        
        self.accuracy = []
        self.accuracy.append(self.w[0])
        
        while np.linalg.norm(grad, ord = 2) > eps:
            if const == 0: 
                start_time = time.time()

                crutch = self.w[0] + beta * (self.w[0] - self.w[1])

                self.w[1] = self.w[0]

                z = sigmoid(X @ crutch)

                grad = self._calc_grad(X, y, z, lambd)

                self.w[0] = crutch - lr * grad

                if len(losses) % 100 == 0:
                    self.accuracy.append(crutch - lr * grad)

                losses.append(np.linalg.norm(grad, ord = 2))

                end_time = time.time()

                time_now += end_time - start_time

                if len(losses) % 100 == 0:
                    self.times.append(time_now)
            else:
                start_time = time.time()

                crutch = self.w[0] + (i / (i + const)) * (self.w[0] - self.w[1])

                self.w[1] = self.w[0]

                z = sigmoid(X @ crutch)

                grad = self._calc_grad(X, y, z, lambd)

                self.w[0] = crutch - lr * grad

                if len(losses) % 10 == 0:
                    self.accuracy.append(crutch - lr * grad)

                losses.append(np.linalg.norm(grad, ord = 2))

                end_time = time.time()

                time_now += end_time - start_time

                if len(losses) % 10 == 0:
                    self.times.append(time_now)
                
                i += 1
                
        return losses
    
    def predict_proba(self, X):
        return sigmoid(X @ self.w[0])
    
    def get_times(self):
        return self.times
    
    def get_accuracy(self, X, y):
        accuracy = []
        for i in range(len(self.accuracy)):
            if i != 0 and accuracy_score(np.round(sigmoid(X @ self.accuracy[i])), y) == 1.0:
                break
            accuracy.append(accuracy_score(np.round(sigmoid(X @ self.accuracy[i])), y))
        
        return accuracy

__(е)__ Решите задачу логистической регресии с помощью метода Нестерова (не забудьте разделить выборку на две части: обучающую и тестовую). Зафиксируйте шаг $\frac{1}{L}$ и перебирайте разные значения моментума от -1 до 1. Проверьте также значения моментума равные $\frac{k}{k+3}$, $\frac{k}{k+2}$, $\frac{k}{k+1}$ ($k$ - номер итерации), а если решаете сильно выпуклую задачу, то и $\frac{\sqrt{L} - \sqrt{\mu}}{\sqrt{L} + \sqrt{\mu}}$. Постройте график сходимости метода от числа итераций (критерий сходимости подберите самостоятельно) для различных значений моментума. Всегда ли сходимость является монотонной?

In [ ]:
#Ищем моменумы
Betas = [-0.5, 0, 0.5, 0, 0, 0]
losses = []
x = []
for i in range(len(Betas)):
    if i < 3:
        regressor = Nesterov()
        losses.append(regressor.fit(X_train, y_train, 1/L, Betas[i], eps = 10**(-2)))
        x.append(np.array(range(len(losses[i]))))
    else: 
        regressor = Nesterov()
        losses.append(regressor.fit(X_train, y_train, 1/L, Betas[i], const = i - 2, eps = 10**(-2)))
        x.append(np.array(range(len(losses[i]))))

In [ ]:
fig = plt.figure(figsize = (15, 7))

for i in range(3):
    plt.plot(x[i][1:], losses[i][1:], label = r'$\beta = $' + str(Betas[i]))
plt.plot(x[3][1:], losses[3][1:], label = r'$\beta = \dfrac{k}{k + 1}$')
plt.plot(x[4][1:], losses[4][1:], label = r'$\beta = \dfrac{k}{k + 2}$')
plt.plot(x[5][1:], losses[5][1:], label = r'$\beta = \dfrac{k}{k + 3}$')

plt.title("Графики сходимости в логарифмическом масштабе")
plt.xlabel("Номер итерации")
plt.ylabel("Значение критерия")

plt.text(1300, 0.5, r'Критерий $||\nabla f\:|| < 10^{-2}$', fontsize = 20)
plt.yscale('log')
plt.legend(loc = 'center right')
plt.grid(alpha = 0.2)
plt.show()

Как видно по последним трем моментумам(особенно отчетливо для красной линии), сходимость действительно может быть немонотонная.

__(ж)__ Для лучшего значения моментума постройте график зависимости точности модели на тестовой выборке от времени работы метода. Добавьте этот график к графикам для тяжелого шарика и градиентного спуска из пункта (г). Сделайте итоговый вывод.

In [ ]:
len(Nest.fit(X_train, y_train, 1/L, 0, const = 1, eps = 10**(-2)))

In [ ]:
Lin_reg = My_Linear_Regression(fit_intercept = False)
Heavy = MyHeavyBall()
Nest = Nesterov()

Lin_reg.fit(X_train, y_train, 0.095725, eps = 10**(-3))
Heavy.fit(X_train, y_train, 1/L, 0.75, eps = 10**(-3))
Nest.fit(X_train, y_train, 1/L, 0, const = 1, eps = 10**(-3))

fig = plt.figure(figsize = (15, 10))

plt.plot(Lin_reg.get_times()[:len(Lin_reg.get_accuracy(X_test, y_test))], Lin_reg.get_accuracy(X_test, y_test), 
         color = 'red', label = 'Обычная линейная регрессия')

plt.plot(Heavy.get_times()[:len(Heavy.get_accuracy(X_test, y_test)[:-1])], Heavy.get_accuracy(X_test, y_test)[:-1],
         color = 'black',  label = 'Тяжелый шарик')

plt.plot(Nest.get_times()[:len(Nest.get_accuracy(X_test, y_test)[:-1])], Nest.get_accuracy(X_test, y_test)[:-1],
         color = 'grey',  label = 'Нестеров')

plt.title("Зависимость accuracy от времени работы")
plt.xlabel("Время, сек")
plt.ylabel("Точность по метрике accuracy_score")

plt.text(6, 0.9, r'Критерий $||\nabla f\:|| < 10^{-3}$', fontsize = 20)
plt.legend(loc = 'center right')
plt.grid(alpha = 0.2)
plt.show()

Вывод: логистическая регрессия решается градиентным спуском хорошо, метод тяжелого шарика улучшает точность решения, а метод нестерова и точность и скорость сходимости.

__Бонусные пункт__

__(з)__ Сделаем подбор константы $L$ адаптивным. Как упоминалось на семинаре, можно измерять локальную $L$, используя:
$$
f(y) \leq f(x^k) + \langle \nabla f(x^k), y - x^k \rangle + \frac{L}{2}\|x^k - y\|_2^2
$$
В частности, может подойти процедура:

```python
def backtracking_L(f, grad, x, h, L0, rho):
    L = L0
    fx = f(x)
    gradx = grad(x)
    while True:
        y = x - 1 / L * h
        if f(y) <= fx - 1 / L gradx.dot(h) + 1 / (2 * L) h.dot(h):
            break
        else:
            L = L * rho
    return L
```

Каким стоит взять __h__? __rho__ должно быть больше или меньше 1? __L0__ надо брать заведомо большим или маленьким?

In [ ]:
#ответ

__(и)__ Поэксперементируйте с этой процедурой, встроенной в подбор $L$ для шага градиентного спуска. В качестве задачи продолжайте рассматривать логистическую регрессию из Задачи 1. Аналогично встройте процедуру подбора $L$ в метод тяжелого шарика и ускоренный метод Нестерова. Постройте график сходимости метода от числа итераций (критерий сходимости подберите самостоятельно). Отобразите на этом графике градиентный спуск, тяжелый шарик и метод Нестерова с адаптивным шагом и шагом $\frac{1}{L}$ (всего 6 линий на графике). Сделайте вывод.

In [ ]:
#ответ

__(к)__ Постройте аналогичный пункту (и) график точности модели от времени.

In [ ]:
#ответ

__(л)__ В [работе](https://arxiv.org/pdf/1204.3982.pdf) представлена техника рестартов для подавления немонотонной сходимости Алгоритма 2 (метод Нестерова). Попробуйте повторить эксперименты авторов на $\ell_2$-регуляризованной квадратичной или логистической регресии. Возьмите параметр регуляризации $\lambda = L / 100$.

In [ ]:
#ответ